https://pytorch.org/tutorials/advanced/cpp_export.html#step-3-loading-your-script-module-in-c

## Converting to Torch Script via Tracing

In [1]:
import torch
import torchvision

# An instance of your model.
model = torchvision.models.resnet18()

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 3, 224, 224)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

In [2]:
output = traced_script_module(torch.ones(1, 3, 224, 224))

In [3]:
output[0, :5]

tensor([-0.2852, -0.3200,  0.1080,  0.4040,  1.0414], grad_fn=<SliceBackward>)

## 1.Converting to Torch Script via Annotation

In [4]:
import torch

class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

由于此模块的forward方法使用依赖于输入的控制流，因此不适合跟踪。相反，我们可以将其转换为ScriptModule。为了将模块转换为ScriptModule，需要使用torch.jit.script编译模块。脚本如下:

If you need to exclude some methods in your nn.Module because they use Python features that TorchScript doesn’t support yet, you could annotate those with @torch.jit.ignore

my_module is an instance of ScriptModule that is ready for serialization.

In [5]:
my_module = MyModule(10,20)
sm = torch.jit.script(my_module)

## 2.Serializing Your Script Module to a File

In [6]:
traced_script_module.save("traced_resnet_model.pt")

This will produce a traced_resnet_model.pt file in your working directory. If you also would like to serialize my_module, call my_module.save("my_module_model.pt") We have now officially left the realm of Python and are ready to cross over to the sphere of C++.

## 3.Loading Your Script Module in C++